In [42]:
from keras.models import load_model
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from nltk import tokenize
from konlpy.tag import Okt
%matplotlib inline
import matplotlib
import matplotlib.font_manager as fm
from sklearn.decomposition import PCA
import nltk
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [18]:
ss_article = pd.read_csv('./삼성전자기사-중앙일보.csv')
ss_article

,datetime,article
0,2020.11.1316:52,삼성전자 주가가 가보지 않은 길에 들어섰다. 1주당 6만3000원을 돌파하며 10개...
1,2020.11.1314:15,삼성전자 주가가 장중 역대 최고가6만 2400원를 뛰어넘었다. ...
2,2020.11.1016:16,삼성전자 서초사옥 〈뉴시스〉 특허청 특별사법경찰팀특사경이 지난 국정감...
3,2020.11.0720:05,전자랜드 유도훈. 사진 KBL 마침내 유도훈 전자랜드 감독이...
4,2020.11.0720:01,인천 전자랜드가 삼성을 꺾고 선두를 유지했다. 유도훈 전자랜드 감독은 단일팀 300...
...,...,...
1210,2018.10.2809:37,【서울뉴시스】삼성전자가 미국 프로농구NBA팀 애틀란타 호크스Atlanta Hawks...
1211,2018.10.2809:37,【서울뉴시스】삼성전자가 미국 프로농구NBA팀 애틀란타 호크스Atlanta Hawks...
1212,2018.10.2809:37,【서울뉴시스】삼성전자가 미국 프로농구NBA팀 애틀란타 호크스Atlanta Hawks...
1213,2018.10.2809:37,【서울뉴시스】삼성전자가 미국 프로농구NBA팀 애틀란타 호크스Atlanta Hawks...


In [28]:
ss_article['datetime'] = ss_article['datetime'].str.replace('.','-')

In [29]:
ss_article

,datetime,article
0,2020-11-1316:52,삼성전자 주가가 가보지 않은 길에 들어섰다. 1주당 6만3000원을 돌파하며 10개...
1,2020-11-1314:15,삼성전자 주가가 장중 역대 최고가6만 2400원를 뛰어넘었다. ...
2,2020-11-1016:16,삼성전자 서초사옥 〈뉴시스〉 특허청 특별사법경찰팀특사경이 지난 국정감...
3,2020-11-0720:05,전자랜드 유도훈. 사진 KBL 마침내 유도훈 전자랜드 감독이...
4,2020-11-0720:01,인천 전자랜드가 삼성을 꺾고 선두를 유지했다. 유도훈 전자랜드 감독은 단일팀 300...
...,...,...
1210,2018-10-2809:37,【서울뉴시스】삼성전자가 미국 프로농구NBA팀 애틀란타 호크스Atlanta Hawks...
1211,2018-10-2809:37,【서울뉴시스】삼성전자가 미국 프로농구NBA팀 애틀란타 호크스Atlanta Hawks...
1212,2018-10-2809:37,【서울뉴시스】삼성전자가 미국 프로농구NBA팀 애틀란타 호크스Atlanta Hawks...
1213,2018-10-2809:37,【서울뉴시스】삼성전자가 미국 프로농구NBA팀 애틀란타 호크스Atlanta Hawks...


In [15]:
ss_bunbong = pd.read_csv('./삼성전자_분당.csv')
ss_bunbong

,date,time,open,high,low,close,volume
0,2018-10-29,09:01,40850,41100,40800,40850,438391
1,2018-10-29,09:02,40800,41000,40800,40950,117568
2,2018-10-29,09:03,40950,41000,40900,41000,58216
3,2018-10-29,09:04,41000,41050,40950,41000,87591
4,2018-10-29,09:05,41050,41050,40750,40750,110698
...,...,...,...,...,...,...,...
190696,2020-11-06,15:17,60000,60100,60000,60100,32567
190697,2020-11-06,15:18,60100,60100,60000,60000,46745
190698,2020-11-06,15:19,60100,60100,60000,60100,61694
190699,2020-11-06,15:20,60000,60100,60000,60100,36475


In [30]:
ss_bunbong['datetime'] = ss_bunbong['date'] +''+ss_bunbong['time']

In [31]:
ss_bunbong

,date,time,open,high,low,close,volume,datetime
0,2018-10-29,09:01,40850,41100,40800,40850,438391,2018-10-2909:01
1,2018-10-29,09:02,40800,41000,40800,40950,117568,2018-10-2909:02
2,2018-10-29,09:03,40950,41000,40900,41000,58216,2018-10-2909:03
3,2018-10-29,09:04,41000,41050,40950,41000,87591,2018-10-2909:04
4,2018-10-29,09:05,41050,41050,40750,40750,110698,2018-10-2909:05
...,...,...,...,...,...,...,...,...
190696,2020-11-06,15:17,60000,60100,60000,60100,32567,2020-11-0615:17
190697,2020-11-06,15:18,60100,60100,60000,60000,46745,2020-11-0615:18
190698,2020-11-06,15:19,60100,60100,60000,60100,61694,2020-11-0615:19
190699,2020-11-06,15:20,60000,60100,60000,60100,36475,2020-11-0615:20


In [49]:
ss_dataset_ = pd.merge(ss_article,ss_bunbong,how='inner',on='datetime')

In [52]:
ss_dataset_ = ss_dataset_.drop(['date','time'],axis=1)

In [53]:
ss_dataset_

,datetime,article,open,high,low,close,volume
0,2020-11-0510:22,연합뉴스 신한은행은 삼성전자와 차세대 인공지능 서비스 개발 협력 및...,59700,59700,59600,59700,14657
1,2020-11-0510:22,연합뉴스 신한은행은 삼성전자와 차세대 인공지능 서비스 개발 협력 및...,59700,59700,59600,59700,14657
2,2020-11-0314:43,김만재사진 왼쪽 금속노련 위원장과 나기홍 삼성전자 경영지원실 인사팀장부사장이 3일 ...,58700,58800,58700,58700,21066
3,2020-11-0314:43,김만재사진 왼쪽 금속노련 위원장과 나기홍 삼성전자 경영지원실 인사팀장부사장이 3일 ...,58700,58800,58700,58700,21066
4,2020-11-0211:38,김기남 삼성전자 대표이사 부회장이 2일 수원 삼성 디지털시티에서 열린 창립 51주년...,57100,57100,57000,57100,4710
...,...,...,...,...,...,...,...
503,2018-10-3011:00,【서울뉴시스】김종민 기자 삼성전자가 픽셀 크기를 절반으로 줄이면서도 밝고 깨끗한 ...,42600,42650,42550,42600,9637
504,2018-10-3009:30,【서울뉴시스】 김정호 기자 코스피 시가총액 1 2위 종목인 삼성전자005930와 ...,42450,42450,42400,42400,29979
505,2018-10-2915:30,기술의 변화와 혁신이 전례 없는 규모와 속도로 이뤄지는 대전환의 시대를 맞고 있습니...,41400,41400,41400,41400,1084329
506,2018-10-2909:56,용인문화재단이사장 백군기이 15일부터 19일까지 삼성전자 화성캠퍼스 DSR타워 로비...,41050,41100,41050,41100,6777


In [54]:
ss_dataset_['prev'] = ss_dataset_['close'] - ss_dataset_['open']

In [92]:
ss_dataset_['sub'] = ''

In [93]:
ss_dataset_article = ss_dataset_['article']

In [94]:
okt = Okt()

In [95]:
# 불용어 처리
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다',",",'"','`','.','에'
            ,'된다','(',')','이다','중','없다','인','것','보인다','밝혔다','것','하는','받았다','바','며','당시','만큼','경우','면서','주기','위해'
            ,'하나로','통해','최근','지난','올해','거저','도중','마지막','로부터','여러','가지','앞서','각각'
            ,'물이','더욱','또한','만들기','오전','현재','초반','오후','무단','배포','금지'
            ,'모든','다른']

In [96]:
cols = []
for i in range(1,21,1):
    col = 'top_'+str(i)
    cols.append(col)

In [97]:
ss_dataset = [] # 필터링용

top20_vector =[]

for a in tqdm(ss_dataset_article):
    
    ss_nouns = okt.nouns(a) # 명사만 추출
#     print(ss_nouns)
    for w in ss_nouns: # 2글자 이상
        if len(w) > 1:
            ss_dataset.append(w)
        else:
            continue
    
#     print(ss_dataset)
    # 불용어 처리
    ss_stopwords = [s for s in ss_dataset if s not in stopwords]
#     print(ss_stopwords)
    # Text화
    ss_text = nltk.Text(ss_stopwords)
    
    
    top20 = ss_text.vocab().most_common(20)
#     print(top20)

    top20_content = [] # top20
    for t in top20:
        
        line = []
        line.append(t[0])
        top20_content.append(line)
#     print(top20_content)
    
    # word2vec 모델 생성
    model = Word2Vec(
        top20_content
        ,size = 300
        ,window =5
        ,min_count=1
        ,workers=1
        ,sg=0
    )
    
    
    # 단어 벡터를 구한다
    word_vectors = model.wv
    vocabs = word_vectors.vocab.keys()
    word_vectors_list = [word_vectors[v] for v in vocabs]
#     print(word_vectors_list)
#     print(len(model.wv.vectors))
    
    # 차원 낮추기
    pca = PCA(n_components=1)
    xys = pca.fit_transform(word_vectors_list)
#     print(len(xys))

    xys = xys.reshape(-1,20)
    
    df = pd.DataFrame(xys,columns=cols)
    top20_vector.append(df)
    
article_vector = pd.concat(top20_vector,ignore_index=True)
    
    
    
    
    
    
    
    
    
# #     for x in xys:
# #         top30_vector.append(list(x))
        
# # article_vector = pd.DataFrame(top30_vector)

    

100%|████████████████████████████████████████████████████████████████████████████████| 508/508 [01:39<00:00,  5.13it/s]


In [98]:
article_vector

,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
0,-0.001781,0.003271,0.004086,-0.001135,0.001501,-0.004835,-0.000966,-0.003646,-0.004613,-0.006358,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
1,-0.001781,0.003271,0.004086,-0.001135,0.001501,-0.004835,-0.000966,-0.003646,-0.004613,-0.006358,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
2,-0.001888,-0.007257,-0.005186,-0.002940,0.009343,-0.004191,0.001636,0.004627,0.008597,0.004124,-0.002892,0.003451,-0.000036,-0.001719,-0.004427,-0.004889,-0.004051,0.003199,0.003208,0.001291
3,-0.005463,-0.005665,-0.006925,-0.001601,-0.003582,0.001123,0.004723,-0.003014,-0.006190,0.002587,0.008042,-0.007350,0.001125,-0.001437,0.003927,0.004096,0.002589,0.005857,0.005085,0.002075
4,-0.004144,-0.004673,-0.007754,-0.002777,-0.002292,-0.000768,0.007042,-0.003737,-0.004452,0.008441,-0.000218,-0.007326,0.000920,-0.001696,0.003371,0.004673,0.003143,0.003143,0.002211,0.006893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,-0.001406,0.000355,0.009120,-0.003419,0.002723,0.008188,-0.005366,-0.004806,0.000798,0.000501,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
504,-0.001406,0.000355,0.009120,-0.003419,0.002723,0.008188,-0.005366,-0.004806,0.000798,0.000501,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,-0.008176,0.005536,0.005423,0.003788,-0.000982
505,-0.001406,0.000355,0.009120,-0.003419,0.002723,0.008188,-0.005366,-0.004806,0.000798,0.000501,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
506,-0.001406,0.000355,0.009120,-0.003419,0.002723,0.008188,-0.005366,-0.004806,0.000798,0.000501,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982


In [99]:
aa = pd.concat([ss_dataset_,article_vector],axis=1)
aa

,datetime,article,open,high,low,close,volume,prev,sub,top_1,...,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
0,2020-11-0510:22,연합뉴스 신한은행은 삼성전자와 차세대 인공지능 서비스 개발 협력 및...,59700,59700,59600,59700,14657,0,,-0.001781,...,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
1,2020-11-0510:22,연합뉴스 신한은행은 삼성전자와 차세대 인공지능 서비스 개발 협력 및...,59700,59700,59600,59700,14657,0,,-0.001781,...,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
2,2020-11-0314:43,김만재사진 왼쪽 금속노련 위원장과 나기홍 삼성전자 경영지원실 인사팀장부사장이 3일 ...,58700,58800,58700,58700,21066,0,,-0.001888,...,-0.002892,0.003451,-0.000036,-0.001719,-0.004427,-0.004889,-0.004051,0.003199,0.003208,0.001291
3,2020-11-0314:43,김만재사진 왼쪽 금속노련 위원장과 나기홍 삼성전자 경영지원실 인사팀장부사장이 3일 ...,58700,58800,58700,58700,21066,0,,-0.005463,...,0.008042,-0.007350,0.001125,-0.001437,0.003927,0.004096,0.002589,0.005857,0.005085,0.002075
4,2020-11-0211:38,김기남 삼성전자 대표이사 부회장이 2일 수원 삼성 디지털시티에서 열린 창립 51주년...,57100,57100,57000,57100,4710,0,,-0.004144,...,-0.000218,-0.007326,0.000920,-0.001696,0.003371,0.004673,0.003143,0.003143,0.002211,0.006893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,2018-10-3011:00,【서울뉴시스】김종민 기자 삼성전자가 픽셀 크기를 절반으로 줄이면서도 밝고 깨끗한 ...,42600,42650,42550,42600,9637,0,,-0.001406,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
504,2018-10-3009:30,【서울뉴시스】 김정호 기자 코스피 시가총액 1 2위 종목인 삼성전자005930와 ...,42450,42450,42400,42400,29979,-50,,-0.001406,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,-0.008176,0.005536,0.005423,0.003788,-0.000982
505,2018-10-2915:30,기술의 변화와 혁신이 전례 없는 규모와 속도로 이뤄지는 대전환의 시대를 맞고 있습니...,41400,41400,41400,41400,1084329,0,,-0.001406,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
506,2018-10-2909:56,용인문화재단이사장 백군기이 15일부터 19일까지 삼성전자 화성캠퍼스 DSR타워 로비...,41050,41100,41050,41100,6777,50,,-0.001406,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982


In [100]:
samsung_data = aa.drop(['article'],axis=1)

In [101]:
samsung_data = samsung_data.set_index('datetime')

In [102]:
samsung_data

,open,high,low,close,volume,prev,sub,top_1,top_2,top_3,...,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
datetime,,,,,,,,,,,,,,,,,,,,,
2020-11-0510:22,59700,59700,59600,59700,14657,0,,-0.001781,0.003271,0.004086,...,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
2020-11-0510:22,59700,59700,59600,59700,14657,0,,-0.001781,0.003271,0.004086,...,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
2020-11-0314:43,58700,58800,58700,58700,21066,0,,-0.001888,-0.007257,-0.005186,...,-0.002892,0.003451,-0.000036,-0.001719,-0.004427,-0.004889,-0.004051,0.003199,0.003208,0.001291
2020-11-0314:43,58700,58800,58700,58700,21066,0,,-0.005463,-0.005665,-0.006925,...,0.008042,-0.007350,0.001125,-0.001437,0.003927,0.004096,0.002589,0.005857,0.005085,0.002075
2020-11-0211:38,57100,57100,57000,57100,4710,0,,-0.004144,-0.004673,-0.007754,...,-0.000218,-0.007326,0.000920,-0.001696,0.003371,0.004673,0.003143,0.003143,0.002211,0.006893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-3011:00,42600,42650,42550,42600,9637,0,,-0.001406,0.000355,0.009120,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
2018-10-3009:30,42450,42450,42400,42400,29979,-50,,-0.001406,0.000355,0.009120,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,-0.008176,0.005536,0.005423,0.003788,-0.000982
2018-10-2915:30,41400,41400,41400,41400,1084329,0,,-0.001406,0.000355,0.009120,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982


In [103]:
samsung_data.to_csv('./중앙일보 삼성기사분봉.csv',encoding='utf-8-sig')

In [64]:
scaler = StandardScaler()
scaler.fit(samsung_data)
samsung_scaled = scaler.transform(samsung_data)

In [65]:
samsung_df_scaled = pd.DataFrame(data=samsung_scaled,columns=samsung_data.columns)

In [66]:
samsung_df_scaled

,open,high,low,close,volume,prev,top_1,top_2,top_3,top_4,...,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
0,2.446861,2.436216,2.440831,2.446898,-0.309124,-0.051742,-0.847730,0.999307,0.298236,-0.279750,...,1.738116,0.154110,-0.748512,-0.764372,1.590797,1.532077,1.822244,0.269987,-0.817811,-1.509980
1,2.446861,2.436216,2.440831,2.446898,-0.309124,-0.051742,-0.847730,0.999307,0.298236,-0.279750,...,1.738116,0.154110,-0.748512,-0.764372,1.590797,1.532077,1.822244,0.269987,-0.817811,-1.509980
2,2.276611,2.283220,2.287324,2.276613,-0.223142,-0.051742,-0.909545,-3.653691,-1.863893,-0.646698,...,-0.401876,0.644389,0.092076,-0.337779,-0.715455,-0.782242,-0.601756,0.620165,1.091561,0.432482
3,2.276611,2.283220,2.287324,2.276613,-0.223142,-0.051742,-2.979288,-2.950287,-2.269549,-0.374530,...,2.153880,-1.431608,0.323722,-0.282481,0.844228,0.944686,0.780314,1.335384,1.631603,0.650809
4,2.004211,1.994226,1.997367,2.004156,-0.442570,-0.051742,-2.215848,-2.511826,-2.462804,-0.613402,...,0.223056,-1.427078,0.282840,-0.333251,0.740377,1.055518,0.895595,0.604963,0.804598,1.992476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,-0.464418,-0.462219,-0.467269,-0.464983,-0.376470,-0.051742,-0.630189,-0.289288,1.472150,-0.743987,...,0.610605,-0.421470,-0.855285,-0.731118,-0.470270,1.221476,-1.460296,1.218585,1.258296,-0.200266
504,-0.489955,-0.496218,-0.492853,-0.499040,-0.103568,-0.990489,-0.630189,-0.289288,1.472150,-0.743987,...,0.610605,-0.421470,-0.855285,-0.731118,-0.470270,-1.414092,1.393595,1.218585,1.258296,-0.200266
505,-0.668718,-0.674714,-0.663416,-0.669326,14.041291,-0.051742,-0.630189,-0.289288,1.472150,-0.743987,...,0.610605,-0.421470,-0.855285,-0.731118,-0.470270,1.221476,-1.460296,1.218585,1.258296,-0.200266
506,-0.728306,-0.725713,-0.723113,-0.720411,-0.414839,0.887005,-0.630189,-0.289288,1.472150,-0.743987,...,0.610605,-0.421470,-0.855285,-0.731118,-0.470270,1.221476,-1.460296,1.218585,1.258296,-0.200266


In [67]:
target = samsung_df_scaled['close']

In [68]:
train = samsung_df_scaled.drop('close',axis=1)

In [69]:
model = load_model('samsung_model.h5')

In [104]:
samsung = pd.read_csv('./중앙일보 삼성기사분봉.csv')

In [105]:
samsung

,datetime,open,high,low,close,volume,prev,sub,top_1,top_2,...,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
0,2020-11-0510:22,59700,59700,59600,59700,14657,0,NaN,-0.001781,0.003271,...,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
1,2020-11-0510:22,59700,59700,59600,59700,14657,0,NaN,-0.001781,0.003271,...,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
2,2020-11-0314:43,58700,58800,58700,58700,21066,0,NaN,-0.001888,-0.007257,...,-0.002892,0.003451,-0.000036,-0.001719,-0.004427,-0.004889,-0.004051,0.003199,0.003208,0.001291
3,2020-11-0314:43,58700,58800,58700,58700,21066,0,NaN,-0.005463,-0.005665,...,0.008042,-0.007350,0.001125,-0.001437,0.003927,0.004096,0.002589,0.005857,0.005085,0.002075
4,2020-11-0211:38,57100,57100,57000,57100,4710,0,NaN,-0.004144,-0.004673,...,-0.000218,-0.007326,0.000920,-0.001696,0.003371,0.004673,0.003143,0.003143,0.002211,0.006893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,2018-10-3011:00,42600,42650,42550,42600,9637,0,NaN,-0.001406,0.000355,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
504,2018-10-3009:30,42450,42450,42400,42400,29979,-50,NaN,-0.001406,0.000355,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,-0.008176,0.005536,0.005423,0.003788,-0.000982
505,2018-10-2915:30,41400,41400,41400,41400,1084329,0,NaN,-0.001406,0.000355,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
506,2018-10-2909:56,41050,41100,41050,41100,6777,50,NaN,-0.001406,0.000355,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982


In [108]:
sub = []
for i in range(508):
    if samsung['close'][i] - samsung['open'][i] > 0:
        s = 1
        sub.append(s)
    else:
        s =0
        sub.append(s)
        
sub


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,


In [109]:
samsung['sub'] = sub

In [110]:
samsung

,datetime,open,high,low,close,volume,prev,sub,top_1,top_2,...,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
0,2020-11-0510:22,59700,59700,59600,59700,14657,0,0,-0.001781,0.003271,...,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
1,2020-11-0510:22,59700,59700,59600,59700,14657,0,0,-0.001781,0.003271,...,0.006264,0.000900,-0.004247,-0.003897,0.007926,0.007152,0.007596,0.001898,-0.003428,-0.005686
2,2020-11-0314:43,58700,58800,58700,58700,21066,0,0,-0.001888,-0.007257,...,-0.002892,0.003451,-0.000036,-0.001719,-0.004427,-0.004889,-0.004051,0.003199,0.003208,0.001291
3,2020-11-0314:43,58700,58800,58700,58700,21066,0,0,-0.005463,-0.005665,...,0.008042,-0.007350,0.001125,-0.001437,0.003927,0.004096,0.002589,0.005857,0.005085,0.002075
4,2020-11-0211:38,57100,57100,57000,57100,4710,0,0,-0.004144,-0.004673,...,-0.000218,-0.007326,0.000920,-0.001696,0.003371,0.004673,0.003143,0.003143,0.002211,0.006893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,2018-10-3011:00,42600,42650,42550,42600,9637,0,0,-0.001406,0.000355,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
504,2018-10-3009:30,42450,42450,42400,42400,29979,-50,0,-0.001406,0.000355,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,-0.008176,0.005536,0.005423,0.003788,-0.000982
505,2018-10-2915:30,41400,41400,41400,41400,1084329,0,0,-0.001406,0.000355,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982
506,2018-10-2909:56,41050,41100,41050,41100,6777,50,1,-0.001406,0.000355,...,0.001440,-0.002094,-0.004782,-0.003728,-0.003114,0.005536,-0.008176,0.005423,0.003788,-0.000982


In [111]:
samsung.to_csv('./중앙일보 삼성기사분봉(sub).csv')